<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej"
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

**Author: Jacek Sroka**



# PDD - LAB 2

Plan

1. Installation
2. Introduction to RDD API
3. Broadcast variables
4. Homework




To **edit the colab** first copy it to your drive with the top bars `File -> Save a copy on drive` option.

## Installation


In [ ]:
!pip install pyspark --quiet
!pip install -U -q PyDrive --quiet
!apt install openjdk-8-jdk-headless &> /dev/null

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

In [ ]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().set('spark.ui.port', '4050').setAppName("mlibs").setMaster("local[*]")
sc = SparkContext.getOrCreate(conf=conf)

Note: pyspark documentation is [here](https://spark.apache.org/docs/3.1.2/api/python/reference/index.html). For example: list of `SparkContext` methods is [here](https://spark.apache.org/docs/3.1.2/api/python/reference/pyspark.html#spark-context-apis).


## Introduction to RDD API
We start with a sample RDD obtained from a local Python collection. The second parameter of `parallelize` specifies how many partitions we want to have and thus allows us to control the parallelism. Usually, it is good to have 4x more partitions than cores, but having partitions with too few elements can make the overhead noticeable. On the other hand for some algorithms, the rate of data transfer over the network can grow quickly with the number of partitions (for example the algorithm could operate on partition pairs).

`cache` suggests to Spark to keep this RDD in cluster memory to speed up future usage.

In [ ]:
input_rdd = sc.parallelize(range(1, 10), 2).cache()

`collect` converts the rdd into local python collection on the driver side.

In [ ]:
input_rdd.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9]

Try actions: `first`, `count`, `take(n)` and `takeSample(withReplacement, num, [seed])`. The documentation can be found [here](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.html).

In [ ]:
# put your code here
print(input_rdd.first())
print(input_rdd.count())
print(input_rdd.take(3))
print(input_rdd.takeSample(True, 6))

1
9
[1, 2, 3]
[7, 1, 6, 6, 8, 9]


Compute the total of the RDD elements with `reduce(func)`.

RDD API with all the actions and transformations can be found [here](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.html#pyspark.RDD).

In [ ]:
# put you code here
input_rdd.reduce(lambda x, y: x+y)

45

RDDs are divided into partitions and distributed on the cluster. We can check how RDD is obtained (lineage), how many partitions there are, and how much memory they use with the `toDebugString` method. PS. `decode("ascii")` part is needed to print in colab only.

In [ ]:
print(input_rdd.toDebugString().decode("ascii"))

(2) PythonRDD[1] at RDD at PythonRDD.scala:53 [Memory Serialized 1x Replicated]
 |       CachedPartitions: 2; MemorySize: 176.0 B; DiskSize: 0.0 B
 |  ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:289 [Memory Serialized 1x Replicated]


Let us create a new RDD by transforming each value of `input_rdd` with the `map` transformation.

In [ ]:
plus1 = input_rdd.map(lambda x: x+1)
plus1.collect()

[2, 3, 4, 5, 6, 7, 8, 9, 10]

In [ ]:
print(plus1.toDebugString().decode("ascii"))

(2) PythonRDD[8] at collect at <ipython-input-9-8bfa8a4c60ee>:2 []
 |  PythonRDD[1] at RDD at PythonRDD.scala:53 []
 |      CachedPartitions: 2; MemorySize: 176.0 B; DiskSize: 0.0 B
 |  ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:289 []


Now try the `toDebugString` on the examples from lab1.

Create RDD from lines of the `sample_data/README.md` file and count the lines that contain the letter `a`. For that, you can use the `filter` transformation. Don't forget to cache the RDD as we will use it later.

In [ ]:
# put your code here
lines_of_text = sc.textFile("sample_data/README.md").cache()
print(lines_of_text.filter(lambda line: 'a' in line).count())

14


Convert the `lines_of_text` RDD to RDD of words by using `flatMap`, then use  `map` and `reduceByKey` to count the number of words in the MapReduce way. Note that this is similar to one of the exercises from lab 1, but here we expect you to use RDD API rather than the MapReduce API we provided last week.

In [ ]:
# put your code here
words = lines_of_text.flatMap(lambda line: line.split())
word_counts = words.map(lambda word: (word, 1)).reduceByKey(lambda x,y: x+y)
word_counts.collect()

[('sample', 2),
 ('datasets', 1),
 ('to', 1),
 ('you', 1),
 ('started.', 1),
 ('*', 3),
 ('`california_housing_data*.csv`', 1),
 ('from', 1),
 ('1990', 1),
 ('more', 1),
 ('information', 1),
 ('available', 1),
 ('https://docs.google.com/document/d/e/2PACX-1vRhYtsvc5eOR2FWNCwaBiKL6suIOrxJig8LcSBbmCbyYsayia_DvPOOBlXZ4CAlQ5nlDD8kTaIDRwrN/pub',
  1),
 ('small', 1),
 ('of', 2),
 ('described', 2),
 ('quartet](https://en.wikipedia.org/wiki/Anscombe%27s_quartet);', 1),
 ('it', 1),
 ('Anscombe,', 1),
 ('F.', 1),
 ('J.', 1),
 ("'Graphs", 1),
 ('American', 1),
 ('Statistician.', 1),
 ('JSTOR', 1),
 ('2682899.', 1),
 ('and', 1),
 ('our', 1),
 ('by', 1),
 ('This', 1),
 ('directory', 1),
 ('includes', 1),
 ('a', 3),
 ('few', 1),
 ('get', 1),
 ('is', 4),
 ('California', 1),
 ('housing', 1),
 ('data', 1),
 ('the', 3),
 ('US', 1),
 ('Census;', 1),
 ('at:', 2),
 ('`mnist_*.csv`', 1),
 ('[MNIST', 1),
 ('database](https://en.wikipedia.org/wiki/MNIST_database),', 1),
 ('which', 1),
 ('http://yann.lecun.com

Rather than processing large datasets, Spark can also distribute computation to many machines/cores. Use the Monte Carlo method to estimate the value of Pi. For that generate $n=4*10^5$ random points in square $[-1;1]\times[-1;1]$ and count the proportion of the ones inside the largest circle contained by this square. Make sure that the computation can be distributed to at least 4 cores.

In [ ]:
import random
random.seed(1)

slices = 4
n = 100000 * slices

xs = range(n)
rdd = sc.parallelize(xs, slices).setName("'Initial rdd'")
sample = rdd.map(lambda i: (random.uniform(-1,1), random.uniform(-1,1))).setName("'Random points sample'")
inside_circle = sample.filter(lambda tup: tup[0]**2+tup[1]**2 <= 1).setName("'Random points inside circle'")
count = inside_circle.count()

f"Pi is roughly {4.0 * count / n}"

'Pi is roughly 3.1391'

## Broadcast variables
Did you notice that the `random` value was available to the lambda in `map` even though this takes place on worker nodes while the random singleton is initialized in the driver program? Let us explore this further.

First, let us reinitialize our spark context and make it run more local workers.

In [ ]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().set('spark.ui.port', '4050').setAppName("mlibs").setMaster("local[8]")
sc = SparkContext.getOrCreate(conf=conf)

Now we will create a large local list and reference it in the lambda in the map.

In [ ]:
import time

slices = 8  # change me
list_size = 1000000  # change me
times_to_count_len = 50  # change me

list1 = list(range(list_size))

for i in range(3):
   print(f"Iteration {i}")
   print("===========")
   start_time = time.perf_counter()
   observed_sizes = sc.parallelize(range(times_to_count_len), slices).map(lambda x: len(list1))
   # Collect the small RDD so we can print the observed sizes locally.
   print(observed_sizes.glom().collect())
   print(f"Iteration {i} took {time.perf_counter() - start_time} seconds")
   print()

Iteration 0
[[1000000, 1000000, 1000000, 1000000, 1000000, 1000000], [1000000, 1000000, 1000000, 1000000, 1000000, 1000000], [1000000, 1000000, 1000000, 1000000, 1000000, 1000000], [1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000], [1000000, 1000000, 1000000, 1000000, 1000000, 1000000], [1000000, 1000000, 1000000, 1000000, 1000000, 1000000], [1000000, 1000000, 1000000, 1000000, 1000000, 1000000], [1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000]]
Iteration 0 took 2.7856003219999366 seconds

Iteration 1
[[1000000, 1000000, 1000000, 1000000, 1000000, 1000000], [1000000, 1000000, 1000000, 1000000, 1000000, 1000000], [1000000, 1000000, 1000000, 1000000, 1000000, 1000000], [1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000], [1000000, 1000000, 1000000, 1000000, 1000000, 1000000], [1000000, 1000000, 1000000, 1000000, 1000000, 1000000], [1000000, 1000000, 1000000, 1000000, 1000000, 1000000], [1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000

Here the driver program is the bottleneck. The large list has to be transferred from it to all worker nodes. Spark has a mechanism for that called broadcast variables. Compare the running times.

In [ ]:
import time

slices = 8  # change me
list_size = 1000000  # change me
times_to_count_len = 50  # change me

list1 = list(range(list_size))

for i in range(3):
   print(f"Iteration {i}")
   print("===========")
   start_time = time.perf_counter()

   # all changes are in this two lines
   blist1 = sc.broadcast(list1)
   observed_sizes = sc.parallelize(range(times_to_count_len), slices).map(lambda x: len(blist1.value))

   # Collect the small RDD so we can print the observed sizes locally.
   print(observed_sizes.glom().collect())
   print(f"Iteration {i} took {time.perf_counter() - start_time} seconds")
   print()

Iteration 0
[[1000000, 1000000, 1000000, 1000000, 1000000, 1000000], [1000000, 1000000, 1000000, 1000000, 1000000, 1000000], [1000000, 1000000, 1000000, 1000000, 1000000, 1000000], [1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000], [1000000, 1000000, 1000000, 1000000, 1000000, 1000000], [1000000, 1000000, 1000000, 1000000, 1000000, 1000000], [1000000, 1000000, 1000000, 1000000, 1000000, 1000000], [1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000]]
Iteration 0 took 1.4530337850000024 seconds

Iteration 1
[[1000000, 1000000, 1000000, 1000000, 1000000, 1000000], [1000000, 1000000, 1000000, 1000000, 1000000, 1000000], [1000000, 1000000, 1000000, 1000000, 1000000, 1000000], [1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000], [1000000, 1000000, 1000000, 1000000, 1000000, 1000000], [1000000, 1000000, 1000000, 1000000, 1000000, 1000000], [1000000, 1000000, 1000000, 1000000, 1000000, 1000000], [1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000

## Exercises 1 and 2
Implement matrix multiplication with both methods from the lecture. You can use the following sample data to test your Spark solution. For combining two RDDs you can use the `union` transformation. Also use `flatMap`,  `groupByKey`, `aggregateByKey/combineByKey`, `groupByKey`, and `mapValues` for solution 2.

In [ ]:
import numpy as np

def to_sparse(arr):
  for r in range(len(arr)):
    for c in range(len(arr[0])):
      yield ((r,c), arr[r][c])

M = [[1,2,3], [4,5,6], [7,8,9]]
N = [[1,2], [3,4], [5,6]]
MN = np.matmul(M,N)  # [[22,28], [49,64], [76,100]]

print(f"M={list(to_sparse(M))}")
print(f"N={list(to_sparse(N))}")
print(f"MN={list(to_sparse(MN))}")

M=[((0, 0), 1), ((0, 1), 2), ((0, 2), 3), ((1, 0), 4), ((1, 1), 5), ((1, 2), 6), ((2, 0), 7), ((2, 1), 8), ((2, 2), 9)]
N=[((0, 0), 1), ((0, 1), 2), ((1, 0), 3), ((1, 1), 4), ((2, 0), 5), ((2, 1), 6)]
MN=[((0, 0), 22), ((0, 1), 28), ((1, 0), 49), ((1, 1), 64), ((2, 0), 76), ((2, 1), 100)]


In [ ]:
from collections import defaultdict
I = len(M)
J = len(M[0])  # =len(N)
K = len(N[0])

Mrdd = sc.parallelize(to_sparse(M))
Nrdd = sc.parallelize(to_sparse(N))

# put your code here (using solution 1)
unionrdd = Mrdd.map(lambda x : (x[0][1], (x[0][0], x[1], "M"))).union\
        (Nrdd.map(lambda x : (x[0][0], (x[0][1], x[1], "N")))).groupByKey()



def solution1(tup):
  j, triples = tup
  rel_to_tuples = defaultdict(list)

  for i_or_k, v, rel in triples:
    rel_to_tuples[rel].append((i_or_k,v))
  res = []

  for i, v in rel_to_tuples["M"]:
    for k, w in rel_to_tuples["N"]:
      res.append(((i,k), v*w))

  return res


unionrdd.flatMap(solution1).aggregateByKey(0, lambda x,y : x+y, lambda x,y: x+y).collect()

[((1, 1), 64),
 ((2, 0), 76),
 ((0, 1), 28),
 ((0, 0), 22),
 ((1, 0), 49),
 ((2, 1), 100)]

In [ ]:
I = len(M)
J = len(M[0])  # =len(N)
K = len(N[0])

Mrdd = sc.parallelize(to_sparse(M))
Nrdd = sc.parallelize(to_sparse(N))

Mrdd_multipled = Mrdd.flatMap(lambda t : list( ((t[0][0], k), (t[0][1], t[1]))\
                                              for k in range(K)))
Nrdd_multipled = Nrdd.flatMap(lambda t : list( ((i, t[0][1]), (t[0][0], t[1]))\
                                              for i in range(I)))

unionrdd = Mrdd_multipled.union(Nrdd_multipled).groupByKey()

# put your code here (using solution 2)
def solution2(values_with_j):
  products_for_j = defaultdict(lambda: 1.0)
  for j, v in values_with_j:
    products_for_j[j] *= v
  return sum(products_for_j.values())

unionrdd.mapValues(solution2).collect()

[((1, 1), 64.0),
 ((2, 0), 76.0),
 ((0, 1), 28.0),
 ((0, 0), 22.0),
 ((1, 0), 49.0),
 ((2, 1), 100.0)]

## Exercise 3
Your goal is to join two collections by key. One collection (A) is "big data" size and has to be processed on many machines and the other one (B) is small enough to fit in memory of a single machine. Implement the join by using RDD with the first collection and a broadcast variable with the second one.

In [ ]:
A = [(1,2), (1,3), (2,2)]  # A(X,Y)
B = [(2,4), (3,4), (3,5)]  # B(Y,Z)
AjoinB = [(1,2,4), (1,3,4), (1,3,5), (2,2,4)]  # A(X,Y) join B(Y,Z) on Y

print(f"A={A}")
print(f"B={B}")
print(f"AjoinB={AjoinB}")

A=[(1, 2), (1, 3), (2, 2)]
B=[(2, 4), (3, 4), (3, 5)]
AjoinB=[(1, 2, 4), (1, 3, 4), (1, 3, 5), (2, 2, 4)]


In [ ]:
# put your code here
rddA = sc.parallelize(A)

B_dict = {}
for k, v in B:
  B_dict.setdefault(k, []).append(v)

broadcast_B_dict = sc.broadcast(B_dict)

def solution3(tup):
  x, y = tup
  vs = broadcast_B_dict.value.get(y, [])
  return [(x, y, v) for v in vs]

A_join_B = rddA.flatMap(solution3).collect()

print(f"AjoinB={A_join_B}")

AjoinB=[(1, 2, 4), (1, 3, 4), (1, 3, 5), (2, 2, 4)]


<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>